In [ ]:
import sys # for automation and parallelisation
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])

In [ ]:
import numpy as np
import pandas as pd
from quetzal.model import stepmodel
from quetzal.io import excel

# Inner-zonal assignment
## Saves transport activity within zones
## Needs inner-zonal volumes and modal split

In [ ]:
input_path = '../input/'
output_path = '../output/' + scenario + '/'
model_path = '../model/' + scenario + '/'

In [ ]:
# Load scenario parameters
params = excel.read_var(file='../input/parameters.xls', scenario=scenario)

In [ ]:
try:
    sm = stepmodel.read_json(model_path + 'de_zones')
except FileNotFoundError:
    sm = stepmodel.read_json(model_path.replace(scenario, 'base') + 'de_zones')

In [ ]:
od_cols = ['origin', 'destination']
v_choice = stepmodel.read_zippedpickles(model_path + 'de_volumes_choice')
v_distr = stepmodel.read_zippedpickles(model_path + 'de_volumes_distribution')
sm.volumes = v_choice.volumes.merge(
    v_distr.volumes,
    on=od_cols, how='outer'
).reset_index().fillna(0)

In [ ]:
# Reshape volumes for inner-zonal trips
sm.volumes = sm.volumes.loc[sm.volumes['origin']==sm.volumes['destination']
                           ].drop(['destination', 'index', 'level_0'], axis=1, errors='ignore'
                                 ).set_index('origin', drop=True)
segments = list(sm.volumes.columns)

## Modal split and pkm

In [ ]:
print('Trip volumes by mode')
# Calculate modal split from parameters
sm.volumes.columns = pd.MultiIndex.from_tuples([(c, 0) for c in sm.volumes.columns])
sm.volumes.columns.names = ['segment', 'mode']
modes = [m.strip() for m in params['inner-zonal']['modes'].split(';')]
for seg in segments:
    for mode in modes:
        sm.volumes[(seg+'_volume', mode)] = 0
sm.volumes = sm.volumes.astype(float)
for u in sm.zones['urbanisation'].unique():
    mask = sm.zones['urbanisation']==u
    for seg in segments:
        for mode in modes:
            sm.volumes.loc[mask, (seg+'_volume', mode)] = \
                sm.volumes.loc[mask, (seg, 0)] * params['inner-zonal'][str(u)+'_'+seg+'_'+mode]
sm.volumes.drop([(s, 0) for s in segments], axis=1, inplace=True)
print('Billion trips per year: ', np.round(sm.volumes.sum().unstack().sum() / 1e9, 2))

In [ ]:
print('Passenger kilometres')
# Calculate inner-zonal mean distance through zone size
for seg in segments:
    for mode in modes:
        dist = np.power(sm.zones['area'], params['inner-zonal']['distance_factor_'+mode])
        sm.volumes[(seg+'_pkm', mode)] = dist * sm.volumes[(seg+'_volume', mode)]
        sm.volumes[(seg+'_dist', mode)] = dist
pkm_cols = [(seg+'_pkm', mode) for mode in modes for seg in segments]
print(sm.volumes[pkm_cols].sum().unstack(level=1).sum() / 1e9)

In [ ]:
print('Pkm per car: {}'.format(sm.volumes[pkm_cols].sum().unstack(level=1).sum().loc['car']
                               / params['vehicles']['car'] / params['car_occ']['all']))

In [ ]:
print('Mean distances')
dist_cols = [(seg+'_dist', mode) for mode in modes for seg in segments]
print(sm.volumes[dist_cols].mean().unstack(level=1).mean())

### Generate time

Average duration of a trip

In [ ]:
t_map = {}
t_map['rail'] = {1: 30, 2: 30, 3: 30}
t_map['bus'] = {1: 30, 2: 30, 3: 30}
t_map['car'] = {1: 30, 2: 30, 3: 30}
t_map['non-motorised'] = {1: 10, 2: 15, 3: 15}
for seg in segments:
    for mode in modes:
        sm.volumes[(seg+'_time', mode)] = sm.volumes[(seg+'_dist', mode)] \
            / sm.zones.loc[sm.volumes.index, 'urbanisation'].map(t_map[mode])

In [ ]:
print('Mean durations in minutes')
time_cols = [(seg+'_time', mode) for mode in modes for seg in segments]
print(sm.volumes[time_cols].mean().unstack(level=1).mean() * 60)

### Generate prices

See cal11 for values and explanation

In [ ]:
vc_car = params['car_var_cost']
fix_car = params['car_fix_cost']
parking_cost = params['parking_cost']
n = params['car_occ']['all']
for seg in segments:
    sm.volumes[(seg+'_price', 'rail')] = params['rail_short']['min'] \
        * params['rail_short']['reduction_share_'+seg]
    sm.volumes[(seg+'_price', 'bus')] = sm.zones.loc[sm.volumes.index, 'urbanisation'].map(params['bus']) \
        * params['bus']['multimodal_share'] * params['rail_short']['reduction_share_'+seg]
    sm.volumes[(seg+'_price', 'car')] = [
        (d * vc_car[seg] + parking_cost[u]) / n + fix_car[seg] * t
        for d, u, t in zip(sm.volumes[(seg+'_dist', 'car')],
                           sm.zones.loc[sm.volumes.index, 'urbanisation'],
                           sm.volumes[(seg+'_time', 'car')])]
    sm.volumes[(seg+'_price', 'non-motorised')] = 0

In [ ]:
print('Mean prices')
price_cols = [(seg+'_price', mode) for mode in modes for seg in segments]
print(sm.volumes[price_cols].mean().unstack(level=1).mean())

## Save to excel

Merge volumes and averages into a suitable output format. It should contain columns for
* volumes
* passenger kilometer
* vehicle kilometer
* av. distance
* av. time
* av. price

In [ ]:
df = sm.volumes[[(seg+'_volume', mode) for mode in modes for seg in segments]
               ].stack(level=1).stack().rename('volumes').reset_index(level=[1,2])
df['segment'] = df['segment'].str[:-7]
df['mode'] = df['mode'].replace({'non-motorised': 'walk'})
df['urb'] = sm.zones.loc[df.index, 'urbanisation']
df['length'] = sm.volumes[dist_cols].stack(level=1).stack().reset_index(level=[1,2])[0]
df['time'] = sm.volumes[time_cols].stack(level=1).stack().reset_index(level=[1,2])[0]
df['price'] = sm.volumes[price_cols].stack(level=1).stack().reset_index(level=[1,2])[0]
df['pkm'] = sm.volumes[pkm_cols].stack(level=1).stack().reset_index(level=[1,2])[0]
df['vkm'] = df['pkm'] * n * (df['mode']=='car').astype(int)

In [ ]:
# Create an aggregated version (NUTS2-level and no segments)
df['NUTS1'] = df.index.map(sm.zones['NUTS_ID']).str[:3]
def weighted_average(data, data_col, weight_col, by_col):
    data['_data_times_weight'] = data[data_col] * data[weight_col]
    data['_weight_where_notnull'] = data[weight_col] * pd.notnull(data[data_col])
    g = data.groupby(by_col)
    result = g['_data_times_weight'].sum() / g['_weight_where_notnull'].sum()
    del data['_data_times_weight'], data['_weight_where_notnull']
    return result
df_agg = df.groupby(['NUTS1', 'mode']).agg({'volumes': 'sum', 'pkm': 'sum', 'vkm': 'sum'})
for col in ['length', 'time', 'price']:
    df_agg[col] = weighted_average(df, col, 'volumes', ['NUTS1', 'mode'])

In [ ]:
# Save an excel
with pd.ExcelWriter(output_path + 'inner_zone.xlsx') as writer:
    df.to_excel(writer, sheet_name='inner_zone')
    df_agg.to_excel(writer, sheet_name='agg')

In [ ]:
# Save disaggregated pkm as CSV
sm.volumes[pkm_cols].to_csv(output_path + 'inner_zone_pkm.csv')